## Required Libraries

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf
from pandas.tseries.offsets import BDay
from scipy.optimize import minimize
import openpyxl

## Custom Functions for Data ETL

In [18]:
# Function to download stock data
def download_stock_data(ticker, start_date, end_date):
    try:
        return yf.download(ticker, start=start_date, end=end_date)
    except:
        return None

# Function to calculate price metrics for each stock
def calculate_price_metrics(data, stock_tickers, stock_category, stock_quantity, stock_beta, num_trading_days):
    metrics = []
    data['Daily Return'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
    initial_value = data['Adj Close'].iloc[0]
    final_value = data['Adj Close'].iloc[-1]
    for ticker in stock_tickers:
        metric = {}
        metric['Stock'] = ticker
        metric['Category'] = stock_category
        metric['Net Return'] = (final_value - initial_value) / initial_value
        metric['Volatility'] = data['Daily Return'].std() * np.sqrt(num_trading_days)
        metric['Max Drawdown'] = (1 - data['Adj Close'] / data['Adj Close'].cummax()).max()
        metric['Avg Buy Price'] = initial_value
        metric['Current Price'] = final_value
        metric['Quantity'] = stock_quantity
        metric['Buy Value'] = metric['Avg Buy Price'] * metric['Quantity']
        metric['Current Value'] = metric['Current Price'] * metric['Quantity']
        metric['P/L'] = metric['Current Value'] - metric['Buy Value']
        metric['Beta'] = stock_beta
        metrics.append(metric)
    return metrics

# Function to calculate portfolio statistics
def calculate_portfolio_stats(data, returns_df, num_trading_days, weights, rf_return, benchmark_return):
    stats = {}
    total_investment = np.sum(data['Buy Value'])

    total_pl = np.sum(data['P/L'])

    portfolio_return = np.sum(data['Net Return'] * data['Buy Weights'])
    optimal_return_sr = np.sum(data['Net Return'] * data['Optimal Sharpe'])
    optimal_return_al = np.sum(data['Net Return'] * data['Optimal Alpha'])
    optimal_return_ty = np.sum(data['Net Return'] * data['Optimal Treynor'])
    optimal_pl_sr = total_investment*optimal_return_sr
    optimal_pl_al = total_investment*optimal_return_al
    optimal_pl_ty = total_investment*optimal_return_ty

    portfolio_std = np.sqrt(np.dot(weights, np.dot(returns_df.cov(), weights)) * num_trading_days)
    optimal_std_sr = np.sqrt(np.dot(data['Optimal Sharpe'].T, np.dot(returns_df.cov(), data['Optimal Sharpe'])) * num_trading_days)
    optimal_std_al = np.sqrt(np.dot(data['Optimal Alpha'].T, np.dot(returns_df.cov(), data['Optimal Alpha'])) * num_trading_days)
    optimal_std_ty = np.sqrt(np.dot(data['Optimal Treynor'].T, np.dot(returns_df.cov(), data['Optimal Treynor'])) * num_trading_days)

    portfolio_beta = np.sum(data['Beta'] * data['Weights Now'])
    optimal_beta_sr = np.sum(data['Beta'] * data['Optimal Sharpe'])
    optimal_beta_al = np.sum(data['Beta'] * data['Optimal Alpha'])
    optimal_beta_ty = np.sum(data['Beta'] * data['Optimal Treynor'])
    
    rf = rf_return

    Jensen_Alpha = portfolio_return - (rf + portfolio_beta * (benchmark_return - rf))
    optimal_alpha_sr = optimal_return_sr - (rf + optimal_beta_sr * (benchmark_return - rf))
    optimal_alpha_al = optimal_return_al - (rf + optimal_beta_al * (benchmark_return - rf))
    optimal_alpha_ty = optimal_return_ty - (rf + optimal_beta_ty * (benchmark_return - rf))

    Sharpe_Ratio = (portfolio_return - rf) / portfolio_std
    optimal_sharpe_sr = (optimal_return_sr - rf) / optimal_std_sr
    optimal_sharpe_al = (optimal_return_al - rf) / optimal_std_al
    optimal_sharpe_ty = (optimal_return_ty - rf) / optimal_std_ty

    Treynor_Ratio = (portfolio_return - rf) / portfolio_beta
    optimal_treynor_sr = (optimal_return_sr - rf) / optimal_beta_sr
    optimal_treynor_al = (optimal_return_al - rf) / optimal_beta_al
    optimal_treynor_ty = (optimal_return_ty - rf) / optimal_beta_ty


    stats['Portfolio Statistics'] = ['Total Investment', 'Total P/L', 'Portfolio Return', 'Portfolio Risk', 'Portfolio Beta', 
                                     'Risk-Free Rate', 'Benchmark Return', 'Jensen Alpha', 'Sharpe Ratio', 'Treynor Ratio']
    stats['Current Values'] = [total_investment, total_pl, portfolio_return, portfolio_std, portfolio_beta, rf_return, 
                       benchmark_return, Jensen_Alpha, Sharpe_Ratio, Treynor_Ratio]
    stats['Sharpe Max.'] = [total_investment, optimal_pl_sr, optimal_return_sr, optimal_std_sr, optimal_beta_sr, rf_return,
                               benchmark_return, optimal_alpha_sr, optimal_sharpe_sr, optimal_treynor_sr]
    stats['Alpha Max.'] = [total_investment, optimal_pl_al, optimal_return_al, optimal_std_al, optimal_beta_al, rf_return,
                                benchmark_return, optimal_alpha_al, optimal_sharpe_al, optimal_treynor_al]
    stats['Treynor Max.'] = [total_investment, optimal_pl_ty, optimal_return_ty, optimal_std_ty, optimal_beta_ty, rf_return,
                                benchmark_return, optimal_alpha_ty, optimal_sharpe_ty, optimal_treynor_ty]
    return stats

## User Portfolio Details

In [7]:
input_df = pd.read_excel('PortfolioManagement.xlsx', sheet_name='Universe')

client_name = input("Enter client name: ")
stock_tickers = input("Enter stock tickers: ").split()
stock_categories = input("Enter stock categories: ").split()
stock_quantities = [float(qty) for qty in input("Enter stock quantities: ").split()]
stock_beta = [float(beta) for beta in input("Enter stock beta: ").split()]
benchmark_stock = input("Enter benchmark stock yfinance ticker: ")
rf_return = float(input("Enter risk-free rate of return: "))
start_date_input = input("Enter Start Date for portfolio in yyyy-mm-dd format: ")

# Parse start date input
try:
    start_date = datetime.strptime(start_date_input, "%Y-%m-%d")
except ValueError:
    print("Incorrect format. Please enter the date in yyyy-mm-dd format.")

end_date = datetime.now().strftime('%Y-%m-%d')
num_trading_days = len(pd.date_range(start_date, end_date, freq=BDay()))

# Dictionary to store stock data and metrics
stock_data = {}
stock_metrics = {}

## Portfolio Management

In [8]:
# Loop through stock tickers
for ticker, category, quantity, beta in zip(stock_tickers, stock_categories, stock_quantities, stock_beta):
    data = download_stock_data(ticker + ".NS", start_date, end_date)
    stock_data[ticker] = data
    if data is not None and not data.empty:
        stock_metrics[ticker] = calculate_price_metrics(data, [ticker], category, quantity, beta, num_trading_days)
    else:
        stock_metrics[ticker] = None

# Create a DataFrame to store metrics for each ticker
master_df = pd.DataFrame()

for ticker, data in stock_data.items():
    if data is not None and not data.empty:
        metrics_df = pd.DataFrame(stock_metrics[ticker])
        master_df = pd.concat([master_df, metrics_df], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
# Calculate the total buy value and total current value across the portfolio
total_buy_value = master_df['Buy Value'].sum()
total_current_value = master_df['Current Value'].sum()

# Calculate weights based on Buy Value and Current Value
master_df['Buy Weights'] = master_df['Buy Value'] / total_buy_value
master_df['Weights Now'] = master_df['Current Value'] / total_current_value

# Calculate portfolio statistics
stock_returns = [data['Daily Return'].tolist() for data in stock_data.values() if data is not None and not data.empty]
returns_df = pd.DataFrame(stock_returns).T

# Use stock weights in the portfolio risk calculation
stock_weights = master_df.set_index('Stock')['Buy Weights'].to_dict()
weights = np.array([value for value in stock_weights.values()])

# Fetching benchmark stock data
benchmark_data = yf.download(benchmark_stock, start=start_date)
benchmark_return = (benchmark_data["Adj Close"][-1] - benchmark_data["Adj Close"][0]) / benchmark_data["Adj Close"][0]

[*********************100%%**********************]  1 of 1 completed


## Creating Optimal Portfolio

In [19]:
# Define the objective function (negative Sharpe ratio)
def sharpe_max(weights, returns, num_trading_days, risk_free_rate):
    portfolio_return = np.sum(returns.mean() * weights)*num_trading_days
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights))*num_trading_days)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return -sharpe_ratio

def alpha_max(weights, returns, num_trading_days, risk_free_rate, benchmark_return):
    portfolio_return = np.sum(returns.mean() * weights)*num_trading_days
    portfolio_beta = np.sum(master_df['Beta'] * weights)
    alpha = portfolio_return - (risk_free_rate + portfolio_beta * (benchmark_return - risk_free_rate))
    return -alpha

def treynor_max(weights, returns, num_trading_days, risk_free_rate):
    portfolio_return = np.sum(returns.mean() * weights)*num_trading_days
    portfolio_beta = np.sum(master_df['Beta'] * weights)
    treynor_ratio = (portfolio_return - risk_free_rate) / portfolio_beta
    return -treynor_ratio

def beta_min(weights, returns, num_trading_days, risk_free_rate, benchmark_return):
    portfolio_beta = np.sum(master_df['Beta'] * weights)
    return portfolio_beta

# Define the constraints
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Set initial weights 
initial_weights = master_df['Weights Now'].to_list()

# Define bounds for each variable (weight)
bounds = [(0, 0.3) for _ in range(len(initial_weights))]

# Run optimization for Sharpe ratio
result_sharpe = minimize(sharpe_max, initial_weights, args=(returns_df, num_trading_days, rf_return), method='SLSQP', bounds=bounds, constraints=constraints)

# Run optimization for alpha
result_alpha = minimize(alpha_max, initial_weights, args=(returns_df, num_trading_days, rf_return, benchmark_return), method='SLSQP', bounds=bounds, constraints=constraints)

# Run optimization for Treynor ratio
result_treynor = minimize(treynor_max, initial_weights, args=(returns_df, num_trading_days, rf_return), method='SLSQP', bounds=bounds, constraints=constraints)

# Extract optimized weights
optimal_sharpe = result_sharpe.x
optimal_alpha = result_alpha.x
optimal_treynor = result_treynor.x

master_df['Optimal Sharpe'] = optimal_sharpe
master_df['Optimal Alpha'] = optimal_alpha
master_df['Optimal Treynor'] = optimal_treynor

# Calculate portfolio statistics
portfolio_stats = calculate_portfolio_stats(master_df, returns_df, num_trading_days, initial_weights, rf_return, benchmark_return)

## Saving the client's details in a new sheet

In [20]:
# Save the data to an Excel file
price_file_path = "client_portfolio_analysis.xlsx"

# Load workbook
wb = openpyxl.load_workbook(price_file_path)
ws = wb.create_sheet(title=client_name) if client_name not in wb.sheetnames else wb[client_name]

# Write column headers and data for master_df
for r_idx, (header, *data) in enumerate([master_df.columns.tolist()] + master_df.values.tolist(), start=1):
    for c_idx, value in enumerate([header] + data, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Add a gap of 3 empty rows
start_row = len(master_df) + 4

# Write column headers and data for portfolio_stats
for r_idx, (header, *data) in enumerate([pd.DataFrame(portfolio_stats).columns.tolist()] + pd.DataFrame(portfolio_stats).values.tolist(), start=start_row):
    for c_idx, value in enumerate([header] + data, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Save workbook
wb.save(price_file_path)
print("Data successfully saved to client_portfolio_analysis.xlsx")

Data successfully saved to client_portfolio_analysis.xlsx


In [ ]:
# Define the format dictionary as you've specified
format_dict = {
    'Net Return': '{:.2%}',
    'Volatility': '{:.2%}',
    'Max Drawdown': '{:.2%}',
    'Avg Buy Price': '{:.2f}',
    'Current Price': '{:.2f}',
    'Buy Value': '{:.2f}',
    'Current Value': '{:.2f}',
    'P/L': '{:.2f}',
    'Beta': '{:.2f}',
    'Buy Weights': '{:.2%}',
    'Weights Now': '{:.2%}',
    'Optimal Sharpe': '{:.2%}',
    'Optimal Alpha': '{:.2%}',
    'Optimal Treynor': '{:.2%}',
}

# Reapply the style formatting to the DataFrame
master_df_styled = master_df.style.format(format_dict)

# Now you can display or work with 'master_df_styled' as your styled DataFrame
master_df_styled

,Stock,Category,Net Return,Volatility,Max Drawdown,Avg Buy Price,Current Price,Quantity,Buy Value,Current Value,P/L,Beta,Buy Weights,Weights Now,Optimal Sharpe,Optimal Alpha,Optimal Treynor
0,IREDA,AGG,126.50%,86.65%,39.94%,60.00,135.90,1682.000000,100920.00,228583.79,127663.79,-4.49,4.63%,7.48%,13.93%,30.00%,16.34%
1,HEROMOTOCO,LT,104.54%,25.48%,10.39%,2308.79,4722.30,79.000000,182394.32,373061.68,190667.37,1.00,8.37%,12.21%,26.19%,30.00%,14.70%
2,ONGC,LT,85.97%,25.82%,10.01%,144.14,268.05,1223.000000,176280.77,327825.14,151544.36,1.07,8.09%,10.73%,22.49%,30.00%,12.46%
3,SBILIFE,LT,36.49%,20.60%,7.51%,1099.13,1500.25,164.000000,180256.62,246041.00,65784.38,0.93,8.27%,8.05%,0.00%,0.00%,4.89%
4,RELIANCE,LT,38.61%,18.46%,14.82%,2143.98,2971.70,119.000000,255134.20,353632.29,98498.10,1.01,11.71%,11.58%,0.96%,0.00%,7.02%
5,MARUTI,LT,53.39%,18.42%,8.60%,8214.84,12600.35,20.000000,164296.89,252006.99,87710.10,0.95,7.54%,8.25%,17.93%,10.00%,5.62%
6,PARADEEP,AGG,32.68%,37.78%,20.01%,50.05,66.40,1295.000000,64809.01,85988.00,21178.99,0.82,2.97%,2.81%,2.70%,0.00%,1.71%
7,ICICIBANK,LT,25.65%,15.71%,8.44%,870.09,1093.30,330.000000,287129.04,360789.02,73659.97,1.30,13.17%,11.81%,0.00%,0.00%,7.16%
8,CANFINHOME,AGG,43.01%,32.61%,20.13%,526.49,752.95,126.000000,66337.48,94871.70,28534.22,1.37,3.04%,3.11%,1.15%,0.00%,1.88%
9,DIVGIITTS,AGG,17.37%,41.65%,38.33%,655.98,769.90,116.000000,76093.37,89308.40,13215.04,1.07,3.49%,2.92%,1.35%,0.00%,1.77%


In [ ]:
ret = np.sum(master_df['Net Return']*master_df['Weights Now'])
ret_sh = np.sum(master_df['Net Return']*master_df['Optimal Sharpe'])
ret_al = np.sum(master_df['Net Return']*master_df['Optimal Alpha'])
ret_ty = np.sum(master_df['Net Return']*master_df['Optimal Treynor'])

print(f"Portfolio Return: {ret:.2%}")
print(f"Optimal Sharpe Portfolio Return: {ret_sh:.2%}")
print(f"Optimal Alpha Portfolio Return: {ret_al:.2%}")
print(f"Optimal Treynor Portfolio Return: {ret_ty:.2%}")

Portfolio Return: 49.90%
Optimal Sharpe Portfolio Return: 76.92%
Optimal Alpha Portfolio Return: 100.44%
Optimal Treynor Portfolio Return: 58.25%
